За последнее домашнее задание можно набрать 50 баллов – *таким образом, можно набрать до 5 дополнительных баллов*, которые могут помочь добрать где-либо ранее потерянные баллы, если есть такая необходимость.

**Дедлайн – 24 декабря, 23:59**. *Крайне приветствуется* сдача в ранний срок – работа будет проверена практически сразу (в рабочие часы) и студенту будет выставлены итоговые баллы**


---



**1.** (5 баллов) Вы разработали модель, которая страдает низким смещением и высокой дисперсией. Как Вы думаете, какой метод/алгоритм можно в таком случае применить и почему? 

Для начала надо понять, что является основными источниками высокой дисперсии в финальных предсказаниях модели. В первую очередь, это:
1. шум в данных для обучения модели;
2. использования случайных шагов в алгоритмах обучения (примерами таких моделей может послужить Random Forest или модели на основе стахостического градиентного спуска).


Чтобы понизить дисперсию (variance) модели нам надо увеличить ее смешение (bias). Для этого мы можем, например, обучить модель несколько раз на разных маленьких семплах данных и взять среднию оценку, опираясь на центральную предельную теорему. Для этого мы можем, например, использовать ансамбль моделей (обучить несколько моделей на маленьких смеплах и брать среднее из их предсказаний) или ансамбль параметров (например, обучить на разных маленьких семплах несколько линейных регрессий и взять среднее их весов).  Мы увеличим таким образом смешение, предположив, что среднее значение будет более точным. Второй вариант - это значительно увеличить объем данных для обучения. Здесь мы опираемся на Закон больших чисел.

**2.** (5 баллов) У Вас есть датафрейм `df_baguettes`, содержащий список цен на французские багеты. Но оказалось, что в этом датафрейме отсутствуют многие значения в столбце цен. По крайней мере несколько багетов имеют цену в столбце.
Напишите функцию `median_baguettes`, которая вычисляет медианную цену выбранных французских багетов вместо отсутствующих значений. 

In [9]:
import pandas as pd

def median_baguettes(baguettes):
    baguettes = baguettes.dropna().sort_values('price')
    number_rows = baguettes.shape[0]
    if number_rows % 2 == 0:
        median = (baguettes.iloc[number_rows // 2] + baguettes.iloc[number_rows // 2 - 1]) / 2
    else:
        median = baguettes.iloc[number_rows // 2]

    return baguettes.fillna(median)

*Тесты*

In [10]:
# Even number of elements
df_baguettes = pd.DataFrame([20, None, 32, None, None, 10, 5], columns=['price'])
correct = df_baguettes.fillna(df_baguettes.median()).rename(columns={'price': 'correct price'})
pd.concat([median_baguettes(df_baguettes), correct], axis=1)

,price,correct price
6,5.0,5.0
5,10.0,10.0
0,20.0,20.0
2,32.0,32.0
1,NaN,15.0
3,NaN,15.0
4,NaN,15.0


In [11]:
# Odd number of elements
df_baguettes = pd.DataFrame([20, None, 32, None, None, 5], columns=['price'])
correct = df_baguettes.fillna(df_baguettes.median()).rename(columns={'price': 'correct price'})
pd.concat([median_baguettes(df_baguettes), correct], axis=1)

,price,correct price
5,5.0,5.0
0,20.0,20.0
2,32.0,32.0
1,NaN,20.0
3,NaN,20.0
4,NaN,20.0


In [12]:
# One element
df_baguettes = pd.DataFrame([None,  None, None, 5], columns=['price'])
correct = df_baguettes.fillna(df_baguettes.median()).rename(columns={'price': 'correct price'})
pd.concat([median_baguettes(df_baguettes), correct], axis=1)

,price,correct price
3,5.0,5.0
0,NaN,5.0
1,NaN,5.0
2,NaN,5.0


**3.** (5 баллов) Предположим, что у Вас есть набор данных как с непрерывными, так и с категориальными переменными. Какие методы кластеризации не помогут достичь высокого качества построенной модели при описанной ситуации с данными и почему? И какие методы кластеризации Вы бы использовали? 

Пример модели кластеризации, которая может плохо работать с данными, в которых встречаются категориальные и непрерывные признаки - это KMeans. Это происходит, потому что KMeans плохо работает с признаками из разных шкал, и для нас в целом часто не очень очевидно, как посчитать расстояние между категориальными признаками. Этот эффект можно преодолеть, если предобработать категориальные признаки и граммотно подобрать метрику близости. Например, Евклидова метрика часто бывает не лучшим вариантом для данных с категориальными признаками, выбор метрики сильно зависит от данных, и с ней лучше всего поэксперементировать. Неплохим вариантом метрики может стать Gower Distance. Если вам не подошел KMeans с метрикой Gower Distance, например, из-за большой размерности данных, то стоит попробовать алгоритмы на основе Dimensionality Reduction, например, PCA или его обобщение FAMD (Factorial Analysis of Mixed Data). FAMD отличается предварительной преобработкой данных перед использвоанием PCA, мы сначала используем hot encoded для наших категориальных признаков и нормализуемые числовые признаки (вычитаем среднее и делим на стандартное отклонение). Затем мы так же нормализуем категориальные признаки после hot encoded, но немного по-другому, они делятся на квадрытный корень отношения колличества элементов с этим категориальным признаком ко всем элементам и центрируются (вычитаем среднее значение). Потом запускаем PCA на предобработанных данных. Есть еще ряд техник на основе Dimensionality Reduction, например, UMAP (Uniform Manifold Approximation and Projection for Dimension Reduction). Также мы можем обратиться к модификациям K-Means: K-Modes и K-prototypes. В конечном итоге выбор алгоритма сильно зависит от ваших данных.

**4.** Предположим, что перед Вами стоит задача: построить модель машинного обучения для прогнозирования заполняемости отеля в любую дату.


*   (2.5 баллов) Какие данные Вы бы использовали для обучения вашей модели? 
*   (10 баллов со скелетом кода, 5 баллов с письменным описанием) Какую бы модель Вы разработали? 
*   (2.5 баллов) Как бы Вы оценивали качество модели? 










### Данные
Для прогнозирования заполнености отеля логично использовать данные о времени туристического сезона в стране, где находится отель. Например, в Дубаях туристический сезон - это в основном зимнее время, когда там не так жарко, в Черногорию лучше приезжать в середине Августа. Для этого мы можем ввести категориальный признак для того, какой сейчас сезон, причем необязательно бинарный. Также важным признаком является цена проживания, которая чаще всего меняется для разных дат, и страна. Дополнительно можно добавить несколько бинарных признаков для учета доступности рахных услуг отеля. Например, если это горнолыжный отель, то нам нужно учитывать, есть ли в определенные даты возможность кататься посетителям на лыжах, или она не доступна из-за сезона, при этом одноверменно может быть доступна другая услуга, напрмер, горные прогулки.

### Модель
Так как у нас используются и категориальные, и численные признаки, то нам нужна модель, которая будет работать для смешанных данных. Неплохим вариантом может стать SVM с использованием Direct numerical encoding. Мы будем использовать именно Direct numerical encoding, так как в выбранных нами данных есть порядок, какое значение более приоритетное, чем другое.

### Оценки точности
Классическим вариантом для задач такого типа будет MAE (Mean Absolute Error) и RMSE (Root Mean Squared Error). Первое поможет нам оценить, насколько наши прогнозы далеки от реальных в среднем, а вторая метрика отличается одинаковой шкалой с нашим параметром (что часто удобно для понимания) и поможет нам бороться с большими ошибками. Как мне кажется, RMSE для нас даже более приоритетная метрика, так как небольшие ошибки сильно не навредят нашей бизнес цели. Но если наше предсказание будет давать переодически крупные отклонения, то из-за этого отель может сильно недооценить поток клиентов и не быть готовым или вложить излишние деньги и усилия, когда приедет сильно меньше гостей, чем предполагалось. Также важно учитывать, что если модель ошибается в меньшую сторону, то для нас это лучше, чем если она ошибается в большую, потому что для отеля нехватка мест значительно хуже, чем их избыток. Поэтому имеет смысл рассмотреть применение весов к подсчету MAE и RMSE для того, чтобы придать больший вес положительным оценкам и сместить нашу модель. 

```python
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

# This step Encode categorical values, important to have labels in order.
# Better labels should be in the end of array this will help us to 
# give bigger value for better labels
for feature, feature_labels in categorical_features:
    le = LabelEncoder(feature_labels)
    feature = le.fit(feature)

# Fit our model with standart scaler first
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)

prediction = regr.predict(X_test)

mae = mean_absolute_error(y_test, prediction)
rmse = mean_squared_error(y_test, prediction, squared=False)
```

### Комментарии к коду
Дополнительно стоит рассмотреть вариант использования подхода FAMD для смешанных данных, а также использовать GRidSearchCV для подбора гиперпараметров и тестирования различных ядер.

**5.** (10 баллов) У Вас есть строка, напоминающая объявление списка словарей. Не используя Pandas, напишите функцию `read_split_from_str`, которая:

*   Читает данные и кодирует их как список словарей (данные можно сгенерировать случайно при помощи numpy, например);
*   Разделяет данные на два списка: train и test в соотношении 70:30 и, соответственно, возвращает список `[training_set,testing_set]`. 

Пример входных данных:

`list_of_dict_string = "[{'x': 0, 'y': 4}, {'x': 20, 'y': 104}, {'x': 128, 'y': 212}]"`

Пример выходных данных:

```
def read_split_from_str(list_of_dict_string) 
    [
    [{'x': 0, 'y': 4}, {'x': 20, 'y': 104}],
    [{'x': 128, 'y': 212}]
    ]
```



In [13]:
import re

test = "[{'x': 0, 'y': 4}, {'x': 20, 'y': 104}, {'x': 128, 'y': 212}]"

def read_split_from_str(list_of_dict_string: str):
    """
    Implementation in pure python without any third-party libraries.
    It may be better to use sklearn split for testing and training and np.arrays instead of lists.
    """
    match = re.findall("{.*}", list_of_dict_string)
    dicts = re.split(r"\s*}\s*,\s*{\s*", match[0][1:-1])
    result = []
    for dict_str in dicts:
        dict_obj = {}
        pairs = re.split(r"\s*,\s*", dict_str)
        for pair in pairs:
            try:
                key, value = re.split(r"\s*:\s*", pair)
            except ValueError:
                raise Exception("Incorrect syntax")

            try:
                value = float(value.strip())
            except ValueError:
                pass

            try:
                key = float(key.strip())
            except ValueError:
                key = key.replace("'", '')
            

            dict_obj[key] = value 
        
        result.append(dict_obj)

    n_train = int(len(result)*0.7)
    
    return [result[:n_train], result[n_train:]]


read_split_from_str(test)

[[{'x': 0.0, 'y': 4.0}, {'x': 20.0, 'y': 104.0}], [{'x': 128.0, 'y': 212.0}]]

**6.** (10 баллов) Разработайте модель классификации k-ближайших соседей не используя scikit-learn, соблюдая некоторые условия: 


*   В качестве метрики близости использовано евклидово расстояние;
*   Модель обратаывает датафреймы произвольного количества строк/столбцов.

In [14]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_array, check_X_y, check_is_fitted, check_random_state
from sklearn.utils.multiclass import unique_labels

class ClassificationKNN(BaseEstimator, ClassifierMixin):
    def __init__(self, max_iter=100, random_state=None) -> None:
        self.max_iter = max_iter
        self.random_state = random_state
        super().__init__()

    def get_params(self, deep=True):
        return {
            "max_iter": self.max_iter,
            "random_state": self.random_state
        }

    def set_params(self, **params):
        return super().set_params(**params)

    def compute_distances(self, X):
        distances = np.zeros((X.shape[0], self.num_clusters_))
        for i in range(self.num_clusters_):
            distances[:, i] = np.linalg.norm(X - self.centers_[i, :], ord=2, axis=1)
        return distances

    def compute_centers(self, X) -> bool:
        new_centers = np.zeros((self.num_clusters_, X.shape[1]))
        for i in range(self.num_clusters_):
            new_centers[i, :] = np.mean(X[self.labels_ == i, :], axis=0)
        
        if np.all(new_centers == self.centers_):
            return True
        else:
            self.centers_ = new_centers
            return False

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.num_clusters_ = unique_labels(y).shape[0]

        # init centers
        self.random_state_ = check_random_state(self.random_state)
        rand_indexes = self.random_state_.randint(X.shape[0], size=self.num_clusters_)
        self.centers_ = X[rand_indexes]

        # calculate centers
        for i in range(self.max_iter):
            distances = self.compute_distances(X)
            self.labels_ = np.argmin(distances, axis=1)
            if self.compute_centers(X):
                break

        # map labels from data with clusters centers
        self.map_label_cluster_ = {}
        for i in range(self.num_clusters_):
            labels, count = np.unique(y[self.labels_ == i], return_counts=True)
            self.map_label_cluster_[i] = labels[np.argmax(count)]

    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        distances = self.compute_distances(X)
        centers = np.argmin(distances, axis=1)

        for i in range(centers.shape[0]):
            centers[i] = self.map_label_cluster_[centers[i]]

        return centers

In [15]:
from sklearn import datasets
iris = datasets.load_iris()
X, y = iris.data, iris.target

In [16]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = ClassificationKNN(random_state=42)
clf.fit(X_train, y_train)

prediction = clf.predict(X_test)

accuracy_score(y_test, prediction)

0.9333333333333333